# **Step-1:** Importing Required Libraries

In [ ]:
!pip3 install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.mobilenet import MobileNet
from sklearn.ensemble import VotingClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Step-2:** Performing Preprocessing

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

#Capture training data and labels into respective lists
images = []
labels = []



for directory_path in glob.glob("/content/drive/MyDrive/NIT Durgapur Internship 3/Dataset_Modified/*"):
   data_split=os.path.split(directory_path)
   label=data_split[-1]
   print(label)
   for img_path in glob.glob(os.path.join(directory_path, "*.jpeg")):

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (128, 128))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img=np.asarray(img)
        img=(img/126.5)
        images.append(img)
        labels.append(label)


X_train, X_test, Y_train, Y_test = train_test_split(images, labels, train_size = 0.7, random_state=1, shuffle = True)



Hand
BreastMRI
ChestCT
CXR
AbdomenCT
HeadCT


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded


In [ ]:
y_train[10:20]

In [ ]:
x_train.shape

In [ ]:
 # Normalize pixel values to between 0 and 1

In [ ]:
labels=np.unique(y_test)
labels

In [ ]:
#One hot encode y values for neural network.
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# **Step-3:** Feature Extraction using VGG16

In [ ]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(128,128, 3))

In [ ]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False

VGG_model.summary()  #Trainable parameters will be 0

In [ ]:
#Now, let us use features from convolutional network for RF
feature_extractor_vgg16=VGG_model.predict(x_train)

In [ ]:
fe_vgg16_test=VGG_model.predict(x_test)
#fe_vgg16_test=MobileNet_model.predict(x_test)

In [ ]:
features_vgg16 = feature_extractor_vgg16.reshape(feature_extractor_vgg16.shape[0], -1)

In [ ]:
f_vgg16_test= fe_vgg16_test.reshape(fe_vgg16_test.shape[0],-1)

In [ ]:
print(f_vgg16_test.shape)

# **Step-4:** Classifications using Machine Learning Models

## **Step-4.1:** Classification using K-Nearest Neighbours

In [ ]:
import math
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(features_vgg16 , y_train)
y_pred=knn.predict(f_vgg16_test)

## **Step-4.2:** Classification using Logistic Regression

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
features_vgg16.shape

(18914, 8192)

In [ ]:
# define the multinomial logistic regression model
lrm = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lrm.fit(features_vgg16 , y_train)
y_pred_lr=lrm.predict(f_vgg16_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## **Step-4.3:** Classification using Random Forest

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
RF_model.fit(features_vgg16, y_train) #For sklearn no one hot encoding

RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
#Now predict using the trained RF model.
prediction_RF = RF_model.predict(f_vgg16_test)
#Inverse le transform to get original label back.
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
print(prediction_RF.shape)

(8106,)


In [ ]:
print(test_labels.shape)

(8106,)


## **Step-4.4:** Classification using XGBoost




In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import xgboost as xgb

In [ ]:
#xgb_classifier = xgb.XGBClassifier(tree_method = "gpu_hist")

In [ ]:
xgb_classifier = xgb.XGBClassifier()

In [ ]:
xgb_classifier.fit(features_vgg16, y_train)

In [ ]:
predictions = xgb_classifier.predict(f_vgg16_test)
prediction_XG = le.inverse_transform(predictions)

## **Step-4.5:** Classification using LightGBM


In [ ]:
import lightgbm as lgb

In [ ]:
lightgbm_classifier =lgb.LGBMClassifier()

In [ ]:
lightgbm_classifier.fit(features_vgg16, y_train)

LGBMClassifier()

In [ ]:
predictions = lightgbm_classifier.predict(f_vgg16_test)
prediction_lightgbm = le.inverse_transform(predictions)

## **Step-4.6:** Classification using SVC- Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(C=0.65, random_state=0, kernel='rbf')

In [ ]:
svc.fit(features_vgg16, y_train)

SVC(C=0.65, random_state=0)

In [ ]:
predictions = svc.predict(f_vgg16_test)
prediction_SVC = le.inverse_transform(predictions)

## **Step-6:** Classification using Ensemble Model

### Ensemble Model-1


In [ ]:
final_model = VotingClassifier(estimators=[('rf', RF_model), ('xgb', xgb_classifier),('knn',knn),('svc',svc),('lr',lrm)], voting='weighted')


In [ ]:
final_model.fit(features_vgg16, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=42)),
                             ('xgb', XGBClassifier()),
                             ('knn', KNeighborsClassifier(n_neighbors=10)),
                             ('svc', SVC(C=0.65, random_state=0)),
                             ('lr',
                              LogisticRegression(multi_class='multinomial'))])

In [ ]:
predictions = final_model.predict(f_vgg16_test) # X_test_features
prediction_final = le.inverse_transform(predictions)

### Ensemble Model-2

In [ ]:
final_model_2 = VotingClassifier(estimators=[('rf', RF_model), ('lgbm', lightgbm_classifier),('knn',knn),('svc',svc),('lr',lrm)], voting='hard')


In [ ]:
final_model_2.fit(features_vgg16, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_estimators=50,
                                                     random_state=42)),
                             ('lgbm', LGBMClassifier()),
                             ('knn', KNeighborsClassifier(n_neighbors=10)),
                             ('svc', SVC(C=0.65, random_state=0)),
                             ('lr',
                              LogisticRegression(multi_class='multinomial'))])

In [ ]:
predictions = final_model_2.predict(f_vgg16_test) # X_test_features
prediction_final = le.inverse_transform(predictions)